In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Ingesta Gold") \
    .master("spark://localhost:7077")\
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic")\
    .config("hive.exe.dynamic.partition.mode", "nonstrict") \
    .getOrCreate()

from delta.tables import *

24/07/04 21:29:59 WARN Utils: Your hostname, besgam-Aspire-E1-471 resolves to a loopback address: 127.0.1.1; using 192.168.0.16 instead (on interface wlp9s0)
24/07/04 21:29:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/besgam/.ivy2/cache
The jars for the packages stored in: /home/besgam/.ivy2/jars
:: loading settings :: url = jar:file:/home/besgam/anaconda3/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-13ee751a-617a-442d-848a-e39986336c84;1.0
	confs: [default]
	found io.delta#delta-core_2.12;0.8.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glass

In [22]:
anoMes = "'2023-12'"

In [26]:
consumidor = spark.read.format('delta').load("hdfs://namenode:9000/data/consumidor/silver").where(f"datRefCarga={anoMes}")

In [24]:
df = consumidor.groupBy(col('nomefantasia'), col('datRefCarga'))\
          .agg(count(col('nomefantasia')).alias('qtdReclamcoes'))\
          .orderBy(col('qtdReclamcoes'), ascending=False)\
          .limit(10)

In [25]:
df.write.partitionBy('datRefCarga').format("delta").mode("append").save("hdfs://namenode:9000/data/gold/reclamacaotopten/")

In [27]:
df = consumidor.groupBy(col('nomefantasia'), col('grupoProblema'),col('datRefCarga'))\
          .agg(count(col('nomefantasia')).alias('qtdReclamcoes'))

In [28]:
df.write.partitionBy('datRefCarga').format("delta").mode("append").save("hdfs://namenode:9000/data/gold/grupoproblema/")